# Using PyQIR

PyQIR is a package that provides utilities for generating, parsing, and manipulating QIR.

First, let's use it to generate a simple Bell State program:

In [ ]:
from pyqir import BasicQisBuilder, SimpleModule

# Create the module with two qubits and two results.
bell = SimpleModule("bell", num_qubits=2, num_results=2)
builder = BasicQisBuilder(bell.builder)

# Add instructions to the module to create a Bell pair and measure both qubits.
builder.h(bell.qubits[0])
builder.cx(bell.qubits[0], bell.qubits[1])
builder.mz(bell.qubits[0], bell.results[0])
builder.mz(bell.qubits[1], bell.results[1])

qir = bell.ir()
print(qir)

Since PyQIR can parse and manipulate QIR, it can be used to make simple passes that decompose gates:

In [ ]:
from pyqir import Module, Context, QirModuleVisitor, qis
from math import pi

class DecomposeH(QirModuleVisitor):
    def _on_qis_h(self, call, target):
        # Set the instruction builder to the location just before the call
        # to the H instruction.
        self.builder.insert_before(call)

        # Decompose the H gate into a sequence of RX and RZ gates.
        qis.rx(self.builder, -pi / 2, target)
        qis.rz(self.builder, -pi / 2, target)
        qis.rx(self.builder, -pi / 2, target)

        # Remove the original H instruction.
        call.erase()

module = Module.from_ir(Context(), qir)
DecomposeH().run(module)
print(module)


Or even used to link another module defining functions into the current one:

In [ ]:
decomp_ir = """
%Qubit = type opaque

declare void @__quantum__qis__rx__body(double, %Qubit*)
declare void @__quantum__qis__rz__body(double, %Qubit*)
declare void @__quantum__qis__rzz__body(double, %Qubit*, %Qubit*)

define void @__quantum__qis__cnot__body(%Qubit* %c, %Qubit* %q) alwaysinline {
    call void @__quantum__qis__h__body(%Qubit* %q)
    call void @__quantum__qis__cz__body(%Qubit* %c, %Qubit* %q)
    call void @__quantum__qis__h__body(%Qubit* %q)
    ret void
}

define void @__quantum__qis__cz__body(%Qubit* %c, %Qubit* %q) alwaysinline {
    call void @__quantum__qis__rzz__body(double 1.5707963267948966, %Qubit* %c, %Qubit* %q)
    call void @__quantum__qis__rz__body(double -1.5707963267948966, %Qubit* %c)
    call void @__quantum__qis__rz__body(double -1.5707963267948966, %Qubit* %q)
    ret void
}

define void @__quantum__qis__h__body(%Qubit* %q) alwaysinline {
    call void @__quantum__qis__rx__body(double -1.5707963267948966, %Qubit* %q)
    call void @__quantum__qis__rz__body(double -1.5707963267948966, %Qubit* %q)
    call void @__quantum__qis__rx__body(double -1.5707963267948966, %Qubit* %q)
    ret void
}
"""

# Create a new module from the QIR library using the previous module's context.
decomp_module = Module.from_ir(module.context, decomp_ir)

# Now link the new module to the original module to pull in the function definitions.
module.link(decomp_module)

print(module)

These functions will then be inlined the next time the code is run through any LLVM optimizations.